In [1]:
import requests, time, re
from bs4 import BeautifulSoup
import pandas as pd

# 1. 리뷰 스크래핑

In [2]:
# 잡플래닛 로그인 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

In [3]:
# 1.정부/공공기관/공기업(1001), 2.은행/금융업(900), 3.네트워크/통신/모바일(704), 4.세무/회계(1004)
codes = ['1001', '900', '704', '1004']
nums = ['1', '2', '3', '4']

In [58]:
# PyKoSpacing(띄어쓰기 교정) 패키지 적용
from pykospacing import Spacing
spacing = Spacing()

In [59]:
# 리뷰를 txt파일로 저장
f = open('review_train.txt', 'w', encoding = 'utf-8')

for code, count in zip(codes, nums):
    reviews = {}
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math
    pages = math.ceil(num / 10) # 한 페이지당 리뷰 10개로 구성됨
    
    for i in range(1, pages + 1): # 전체 페이지의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_ = 'us_label') # 한 줄 리뷰 추출

        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().replace('\r', ' ') for label in label[:]]
        
        # 별점이 포함된 div 추출
        star = soup.find_all('div', class_="us_star_m") 
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))

        # txt파일에 리뷰 쓰기
        for k in range(len(labels)):
            start = labels[k].find('"')
            f.write(spacing(labels[k][start+1:-1]) + '  ')
            # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
            reviews[spacing(labels[k][start+1:-1])] = stars[k].replace('.0', '')
 
    # 동적변수를 할당하여 산업군별 리뷰를 각각 리스트로 저장(총 4개의 딕셔너리 생성)
    globals()['reviews' + str(count)] = reviews
    f.write('\n')
f.close()

In [60]:
# 산업군별 리뷰 4개의 딕셔너리를 하나로 합치기
reviews_all = {**reviews1, **reviews2, **reviews3, **reviews4}

In [61]:
len(reviews_all)

2552

In [62]:
# 각 산업군별 리뷰 개수 확인
print(len(reviews1), len(reviews2), len(reviews3), len(reviews4))

774 1062 342 374


In [63]:
with open('review_train.txt', 'r', encoding = 'utf-8') as f:
    print(len(f.readlines()))

4


###  review_train.txt는 하나의 산업군 리뷰 전체를 한 줄로 스크래핑하여 총 4문장으로 이루어져 있음
 - 각 리뷰는 '  '로 구분하여 저장하였음.
 - -> 이는 soynlp의 클래스 사용 시 적합하도록 저장한 것임.
 - (한 줄을 하나의 문서로 인식, 한 문서 내에서 두 칸 띄어쓰기로 문장 구분)
 
### 전체 산업군과 별도로 각각의 산업군별으로도 분석해보기 위해 텍스트 파일을 4개로 분할


In [64]:
with open('review_train.txt', 'r', encoding = 'utf-8') as f :
    data = f.read().split('\n')
    for i in range(4):
        a = 1 + i
        with open(f'review_{a}.txt', 'w+', encoding = 'utf-8') as txt:
            txt.write(data[i]) 

# 2. soynlp를 이용한 명사 추출기 적용
#### 목적: 말뭉치를 기반으로 학습된 기존의 형태소 분석기는 pos 태깅 시 복합명사를 각각의 명사로 태깅하거나 
#### 새로 등장하는 단어를 명사로 인식하지 못하는 문제가 있음. 이에 따라 문서집합의 학습을 통해 명사점수를 부여하고 명사가 아닌지 판단
 - L - [R]의 bipartite graph의 정보를 이용하여 해당 단어가 명사인지 아닌지 판단
 - 해당 패키지는 세종 말뭉치를 이용하여 명사 뒤에 등장하는 R set에서 명사 가능 점수가 학습되어 있음.
 - 명사 가능 점수의 범위: [-1, 1]
 - 통계 기반의 unsupervised 학습 방법 적용
 
#### 두 가지의 명사 추출기를 적용하여 봄

In [65]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

In [66]:
from soynlp.utils import DoublespaceLineCorpus

corpus_fname = 'review_train.txt'
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)
len(sentences)

2557

## 1) LRNounExtractor
#### soynlp의 기본적인 명사 추출기 적용

In [67]:
from soynlp.noun import LRNounExtractor

noun_extractor = LRNounExtractor(
    max_left_length = 10, 
    max_right_length = 7,
    predictor_fnames = None,
    verbose = True
)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded


#### 전체 리뷰 문장에서 단어를 추출하여 명사를 판단하는 학습 실행

In [68]:
%%time
nouns = noun_extractor.train_extract(
    sentences,
    min_noun_score=0.3,
    min_noun_frequency=20
)

[Noun Extractor] scanning was done (L,R) has (651, 327) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 162 nouns are extracted
Wall time: 398 ms


In [69]:
# 빈도수 기준 상위 100개 단어
sort_nouns1 = sorted(nouns.items(), key=lambda x:-x[1].frequency)[:100]
print(sort_nouns1)

[('회사', NounScore_v1(frequency=718, score=0.6732154655172413, known_r_ratio=0.7404255319148936)), ('업무', NounScore_v1(frequency=364, score=0.864112077348066, known_r_ratio=0.9329896907216495)), ('사람', NounScore_v1(frequency=203, score=0.5262195136612021, known_r_ratio=0.6310344827586207)), ('기업', NounScore_v1(frequency=195, score=0.8705182888888888, known_r_ratio=0.6428571428571429)), ('다양', NounScore_v1(frequency=147, score=0.999474713286713, known_r_ratio=0.9727891156462585)), ('사람들', NounScore_v1(frequency=136, score=0.9152982631578946, known_r_ratio=0.9895833333333334)), ('경험', NounScore_v1(frequency=133, score=0.6445116265060242, known_r_ratio=0.7217391304347827)), ('복지', NounScore_v1(frequency=131, score=0.9554130714285715, known_r_ratio=0.875)), ('추천', NounScore_v1(frequency=130, score=0.9863392, known_r_ratio=0.29411764705882354)), ('생각', NounScore_v1(frequency=127, score=0.9078011212121213, known_r_ratio=0.5739130434782609)), ('직원', NounScore_v1(frequency=112, score=0.53233039

In [70]:
# 빈도수 기준 상위 100개 단어 확인
for i, (word, score) in enumerate(sort_nouns1):
    if i % 5 == 0:
        print()
    print('%6s (%.2f)' % (word, score.frequency), end = '')


    회사 (718.00)    업무 (364.00)    사람 (203.00)    기업 (195.00)    다양 (147.00)
   사람들 (136.00)    경험 (133.00)    복지 (131.00)    추천 (130.00)    생각 (127.00)
    직원 (112.00)    연봉 (105.00)   직원들 (101.00)    문화 (100.00)    근무 (98.00)
    직장 (98.00)    성장 (89.00)   안정적 (85.00)    경력 (81.00)    급여 (78.00)
    발전 (71.00)    체계 (71.00)    가능 (66.00)    환경 (62.00)    최고 (59.00)
    좋아 (59.00)   공무원 (58.00)    강도 (57.00)   계약직 (56.00)    하나 (55.00)
    개인 (54.00)    본인 (54.00)    사업 (54.00)    시간 (53.00)    자유 (53.00)
    신입 (53.00)    조직 (52.00)   워라밸 (52.00)    대우 (50.00)    정도 (48.00)
   보수적 (47.00)  공공기관 (47.00)    분들 (45.00)    이상 (44.00)    장점 (44.00)
    무난 (43.00)   사무실 (43.00)    운영 (42.00)    야근 (42.00)    능력 (41.00)
   커리어 (41.00)   수평적 (41.00)    위치 (40.00)    대표 (40.00)   워라벨 (40.00)
    업계 (39.00)    규모 (38.00)    기회 (38.00)    가족 (38.00)   가능성 (38.00)
    부서 (37.00)    노력 (36.00)    느낌 (36.00)    개발 (36.00)   정규직 (36.00)
    필요 (35.00)    수준 (35.00)    단점 (34.00)    자체 (34.00)    입사

In [71]:
# 빈도수 * 명사점수 기준 상위 100개 단어
sort_nouns2 = sorted(nouns.items(), key=lambda x:-x[1].frequency * x[1].score)[:100]
print(sort_nouns2)

[('회사', NounScore_v1(frequency=718, score=0.6732154655172413, known_r_ratio=0.7404255319148936)), ('업무', NounScore_v1(frequency=364, score=0.864112077348066, known_r_ratio=0.9329896907216495)), ('기업', NounScore_v1(frequency=195, score=0.8705182888888888, known_r_ratio=0.6428571428571429)), ('다양', NounScore_v1(frequency=147, score=0.999474713286713, known_r_ratio=0.9727891156462585)), ('추천', NounScore_v1(frequency=130, score=0.9863392, known_r_ratio=0.29411764705882354)), ('복지', NounScore_v1(frequency=131, score=0.9554130714285715, known_r_ratio=0.875)), ('사람들', NounScore_v1(frequency=136, score=0.9152982631578946, known_r_ratio=0.9895833333333334)), ('생각', NounScore_v1(frequency=127, score=0.9078011212121213, known_r_ratio=0.5739130434782609)), ('사람', NounScore_v1(frequency=203, score=0.5262195136612021, known_r_ratio=0.6310344827586207)), ('직원들', NounScore_v1(frequency=101, score=0.8537530862068966, known_r_ratio=1.0)), ('경험', NounScore_v1(frequency=133, score=0.6445116265060242, know

In [72]:
# 빈도수 * 명사점수 기준 상위 100개 단어 확인
for i, (word, score) in enumerate(sort_nouns2):
    if i % 5 ==0:
        print()
    print('%6s (%.2f)' % (word, score.score), end = '')
f.close()


    회사 (0.67)    업무 (0.86)    기업 (0.87)    다양 (1.00)    추천 (0.99)
    복지 (0.96)   사람들 (0.92)    생각 (0.91)    사람 (0.53)   직원들 (0.85)
    경험 (0.64)   안정적 (1.00)    문화 (0.82)    직장 (0.80)    성장 (0.83)
    급여 (0.89)    근무 (0.67)    가능 (0.99)    연봉 (0.62)    체계 (0.86)
    직원 (0.53)    경력 (0.73)    좋아 (0.99)    최고 (0.96)    자유 (0.99)
    본인 (0.95)    신입 (0.92)   공무원 (0.83)    개인 (0.88)    조직 (0.91)
   보수적 (1.00)    환경 (0.76)    분들 (0.97)    무난 (1.00)  공공기관 (0.88)
   커리어 (1.00)   수평적 (1.00)   계약직 (0.72)    업계 (1.00)    운영 (0.93)
   사무실 (0.87)    시간 (0.70)    부서 (1.00)    위치 (0.91)    하나 (0.65)
    규모 (0.95)    야근 (0.85)    대우 (0.70)    필요 (0.99)    느낌 (0.97)
    발전 (0.48)    대표 (0.84)    이상 (0.76)    기회 (0.87)    때문 (1.00)
    노력 (0.92)    부족 (1.00)    입사 (0.99)    강도 (0.57)    능력 (0.77)
    나름 (1.00)    적당 (1.00)    기관 (1.00)    사업 (0.54)   전반적 (1.00)
    전문 (1.00)    관련 (1.00)   공기업 (0.88)    투자 (0.99)    최악 (1.00)
    성과 (1.00)    계속 (1.00)    관심 (1.00)    은행 (0.93)    장점 (0.61)
    구조 (0

## 2) NewsNounExtractor
#### 뉴스 데이터에서 더 좋은 성능을 내기 위해 여러가지 후처리 과정이 추가된 명사 추출기 적용

In [73]:
from soynlp.noun import NewsNounExtractor

noun_extractor = NewsNounExtractor(
    max_left_length=10, 
    max_right_length=7,
    predictor_fnames=None,
    verbose=True
)

used default noun predictor; Sejong corpus based logistic predictor
C:/Users/hsmy1/Anaconda3/envs/py37/lib/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment


In [74]:
nouns = noun_extractor.train_extract(sentences)

scan vocabulary ... 
done (Lset, Rset, Eojeol) = (20648, 10987, 11929)
predicting noun score was done                                        
before postprocessing 3439
_noun_scores_ 1029
checking hardrules ... done / 1029떡볶+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 649
extracted 1 compounds from eojeolss ... 1000 / 1079

In [75]:
# 각 단어별 점수 및 빈도수 확인
words = ['공공기관', '워라밸', '분위기', '야근', '꼰대']

for word in words:
    if not word in nouns:
        continue
    score = nouns[word]
    print('word: %s, score: %.2f, frequency: %.2f' % (word, score.score, score.frequency))

word: 공공기관, score: 0.90, frequency: 62.00
word: 워라밸, score: 0.51, frequency: 52.00
word: 분위기, score: 0.51, frequency: 266.00
word: 야근, score: 0.85, frequency: 48.00
word: 꼰대, score: 0.67, frequency: 29.00


### 각 산업군별 차이를 확인하기 위해 각 산업군별 리뷰 학습 실행

In [76]:
corpus_fname1 = 'review_1.txt'
sentences1 = DoublespaceLineCorpus(corpus_fname1, iter_sent=True)
len(sentences1)

774

In [77]:
corpus_fname2 = 'review_2.txt'
sentences2 = DoublespaceLineCorpus(corpus_fname2, iter_sent=True)
len(sentences2)

1064

In [78]:
corpus_fname3 = 'review_3.txt'
sentences3 = DoublespaceLineCorpus(corpus_fname3, iter_sent=True)
len(sentences3)

345

In [79]:
corpus_fname4 = 'review_4.txt'
sentences4 = DoublespaceLineCorpus(corpus_fname4, iter_sent=True)
len(sentences4)

374

In [80]:
# 리뷰 개수가 적은 산업군을 감안하여 최소 빈도수를 15로 조정
def nounextractor(sent):
    noun_extractor = LRNounExtractor(
    max_left_length = 10, 
    max_right_length = 7,
    predictor_fnames = None,
    verbose = True
    )
    
    nouns = noun_extractor.train_extract(
        sent,
        min_noun_score=0.3,
        min_noun_frequency=15
    )

    # 빈도수 * 명사점수 기준 상위 100개 단어
    sort_nouns = sorted(nouns.items(), key=lambda x:-x[1].frequency * x[1].score)[:100]

    for i, (word, score) in enumerate(sort_nouns):
        if i % 5 ==0:
            print()
        print('%6s (%.2f, %.2f)' % (word, score.frequency, score.score), end = '')

In [81]:
nounextractor(sentences1)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (276, 139) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 59 nouns are extracted

    업무 (130.00, 0.82)   분위기 (73.00, 1.00)    회사 (77.00, 0.53)    다양 (40.00, 1.00)   공무원 (48.00, 0.80)
    복지 (39.00, 0.98)    경험 (56.00, 0.68)    직장 (43.00, 0.87)    근무 (48.00, 0.76)   계약직 (43.00, 0.82)
    기업 (33.00, 1.00)   사람들 (38.00, 0.86)    생각 (37.00, 0.88)  공공기관 (37.00, 0.83)    추천 (30.00, 0.99)
   안정적 (29.00, 1.00)    기관 (26.00, 1.00)    가능 (25.00, 0.99)    급여 (26.00, 0.92)    조직 (26.00, 0.88)
    강도 (24.00, 0.91)   공기업 (22.00, 0.95)    사람 (56.00, 0.37)    환경 (25.00, 0.80)   직원들 (21.00, 0.83)
    좋아 (17.00, 0.98)    경력 (19.00, 0.85)    서울 (16.00, 1.00)    문화 (24.00, 0.66)    만족 (16.00, 0.99)
    체계 (19.00, 0.76)   정규직 (20.00, 0.72)   워라밸 (23.00, 0.62)    관련 (14.00, 1.00)  

In [82]:
nounextractor(sentences2)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (396, 197) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 86 nouns are extracted

    회사 (387.00, 0.66)    업무 (139.00, 0.92)   분위기 (100.00, 1.00)    기업 (98.00, 0.88)   사람들 (64.00, 0.93)
    문화 (58.00, 0.87)    추천 (51.00, 0.98)    다양 (50.00, 1.00)    생각 (52.00, 0.92)    복지 (51.00, 0.94)
    연봉 (67.00, 0.69)    성장 (55.00, 0.83)   안정적 (43.00, 1.00)    사람 (69.00, 0.62)   직원들 (48.00, 0.81)
    최고 (37.00, 0.93)   보수적 (31.00, 1.00)    급여 (33.00, 0.85)    은행 (30.00, 0.93)    체계 (28.00, 0.97)
    본인 (27.00, 0.98)    투자 (26.00, 0.99)    경험 (40.00, 0.63)    직원 (41.00, 0.59)    경력 (33.00, 0.73)
   커리어 (24.00, 1.00)    업계 (24.00, 1.00)    규모 (26.00, 0.92)    직장 (39.00, 0.59)    기회 (23.00, 0.99)
    가능 (23.00, 0.99)    무난 (22.00, 1.00)    조직 (24.00, 0.91)    근무 (31.00, 0.69)

In [83]:
nounextractor(sentences3)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (114, 65) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 16 nouns are extracted

    회사 (144.00, 0.80)    기업 (49.00, 0.71)    업무 (41.00, 0.74)    직원 (31.00, 0.73)    다양 (22.00, 1.00)
    개발 (28.00, 0.75)    복지 (21.00, 0.95)    경험 (19.00, 0.64)    성장 (16.00, 0.69)    생각 (11.00, 0.94)
    사람 (22.00, 0.44)    가능 (9.00, 1.00)    대우 (9.00, 1.00)    연봉 (16.00, 0.47)    사업 (12.00, 0.59)
    근무 (8.00, 0.63)

In [84]:
nounextractor(sentences4)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (128, 70) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 19 nouns are extracted

    회사 (98.00, 0.54)    업무 (50.00, 0.91)    다양 (33.00, 1.00)    사람 (46.00, 0.71)   사무실 (27.00, 1.00)
    신입 (23.00, 0.99)   사람들 (22.00, 0.97)    복지 (20.00, 1.00)   직원들 (19.00, 0.96)    추천 (18.00, 0.99)
    야근 (14.00, 1.00)   회계사 (14.00, 0.98)    경력 (18.00, 0.72)    직원 (30.00, 0.42)   거래처 (16.00, 0.74)
    경험 (18.00, 0.58)    입사 (10.00, 0.99)    생각 (7.00, 0.80)    개인 (12.00, 0.34)